# taking care of the missing value

In [1]:
import pandas as pd
import numpy as np


## its better to fill the missing data instead of deleting it

In [2]:
missing_data = pd.read_csv("car-sales-extended-missing-data.csv")

In [3]:
missing_data.head(20)

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
5,Honda,Red,42652.0,4.0,23883.0
6,Toyota,Blue,163453.0,4.0,8473.0
7,Honda,White,NaN,4.0,20306.0
8,NaN,White,130538.0,4.0,9374.0
9,Honda,Blue,51029.0,4.0,26683.0


## u can notice some places is filled with NaN AKA Missing data

In [4]:
missing_data.isna().sum() #gives how many places missing the value

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [5]:
missing_data.isna().sum() #still the data are missing

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

## filling up data with PANDAS

In [6]:
#filling "Make" column
missing_data["Make"].fillna("missing",inplace=True)

#filling "Colour" column
missing_data["Colour"].fillna("missing",inplace=True)

#fill the "Odometer (KM)" column
missing_data["Odometer (KM)"].fillna(missing_data["Odometer (KM)"].mean(),inplace=True)

missing_data["Doors"].fillna(4,inplace=True)


In [7]:
missing_data.isna().sum()

Make              0
Colour            0
Odometer (KM)     0
Doors             0
Price            50
dtype: int64

## we didnt fill Price becase we want to predict so we are going to delete those row

In [8]:
missing_data.dropna(inplace=True)

In [9]:
missing_data.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

In [10]:
x = missing_data.drop("Price",axis=1)
y = missing_data["Price"]

### no missing data but we lost 50 data

In [11]:
# convert the string to numeric

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_data = ["Make","Colour","Doors"]
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot",one_hot,categorical_data)],
                                  remainder="passthrough")
transformed_x = transformer.fit_transform(x)
transformed_x

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

# FILLING MISSING DATA WITH SCIKIT-LEARN 
## IMPORTED THE SAME DATA SET AGAIN WITH A DIFFERENT NAME

In [12]:
missing_data1 = pd.read_csv("car-sales-extended-missing-data.csv")

In [13]:
missing_data1.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [14]:
missing_data1.dropna(subset=["Price"],inplace=True)
missing_data1.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [15]:
#Split into x and y

x =missing_data1.drop("Price",axis=1)
y=missing_data1["Price"]

In [16]:
# filling the data
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# defining what should be done with the missing and numerical data

cat_imputer = SimpleImputer(strategy="constant",fill_value="missing")
door_imputer = SimpleImputer(strategy="constant",fill_value=4)
num_imputer = SimpleImputer(strategy="mean")

#Define column

cat_feature = ["Make","Colour"]
door_feature = ["Doors"]
num_feature = ["Odometer (KM)"]

#creating imputer

imputer = ColumnTransformer([
    ("cat_imputer",cat_imputer,cat_feature),
    ("door_imputer",door_imputer,door_feature),
    ("num_imputer",num_imputer,num_feature)
])


filled_x = imputer.fit_transform(x)
filled_x

array([['Honda', 'White', 4.0, 35431.0],
       ['BMW', 'Blue', 5.0, 192714.0],
       ['Honda', 'White', 4.0, 84714.0],
       ...,
       ['Nissan', 'Blue', 4.0, 66604.0],
       ['Honda', 'White', 4.0, 215883.0],
       ['Toyota', 'Blue', 4.0, 248360.0]], dtype=object)

In [17]:
new_x=pd.DataFrame(filled_x)
new_x
#if you see here the column values are changed 

,0,1,2,3
0,Honda,White,4.0,35431.0
1,BMW,Blue,5.0,192714.0
2,Honda,White,4.0,84714.0
3,Toyota,White,4.0,154365.0
4,Nissan,Blue,3.0,181577.0
...,...,...,...,...
945,Toyota,Black,4.0,35820.0
946,missing,White,3.0,155144.0
947,Nissan,Blue,4.0,66604.0
948,Honda,White,4.0,215883.0


In [18]:
new_x=pd.DataFrame(filled_x,columns=["Make","Colour","Doors","Odometer (KM)"])
new_x

,Make,Colour,Doors,Odometer (KM)
0,Honda,White,4.0,35431.0
1,BMW,Blue,5.0,192714.0
2,Honda,White,4.0,84714.0
3,Toyota,White,4.0,154365.0
4,Nissan,Blue,3.0,181577.0
...,...,...,...,...
945,Toyota,Black,4.0,35820.0
946,missing,White,3.0,155144.0
947,Nissan,Blue,4.0,66604.0
948,Honda,White,4.0,215883.0


### all the data are filled

In [19]:
new_x.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

## Converting string into number

In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ["Make","Colour","Doors"] #these are categorical not all are string
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot", one_hot, categorical_features)],
                                remainder = "passthrough")

transformed_x = transformer.fit_transform(new_x)
transformed_x

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

# fit the model

In [21]:
np.random.seed(42)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(transformed_x,y,test_size=0.3)


model = RandomForestRegressor()
model.fit(x_train,y_train) 
model.score(x_test,y_test)

0.2396004790354026

### no need worry about the performance 
### our objective is to fill the NaN